In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score,accuracy_score,precision_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier


In [3]:
TP = 355
FP = 1480
FN = 45 
TN = 120

precision  = TP / (TP + FP)  
recall = TP / (TP + FN)    
F1_score = 2 *((precision * recall)/(precision + recall))
print(np.round(F1_score, 4))

0.3177


In [5]:
data=pd.read_csv('Data_for_UCI_named.csv')
data=data.drop(columns=['stab'])

In [6]:
x=data.drop(columns=['stabf'])
y=data['stabf']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)
sc=StandardScaler()
standard_train_df=sc.fit_transform(x_train)
standard_train_df=pd.DataFrame(standard_train_df,columns=x_train.columns)
standard_test_df=sc.transform(x_test)
standard_test_df=pd.DataFrame(standard_test_df,columns=x_test.columns)

In [7]:
rfc=RandomForestClassifier(random_state=1)
rfc.fit(standard_train_df,y_train)
new_predictions=rfc.predict(standard_test_df)
accuracy=accuracy_score(y_true=y_test,y_pred=new_predictions)
print(round(accuracy,4))

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8905


In [8]:
xgb=XGBClassifier(random_state=1,max_depth=3,learning_rate=0.1)
xgb.fit(standard_train_df,y_train)
new_predictions=xgb.predict(standard_test_df)
accuracy=accuracy_score(y_true=y_test,y_pred=new_predictions)
print(round(accuracy,4))

0.9195


In [9]:
lgbm=LGBMClassifier(random_state=1)
lgbm.fit(standard_train_df,y_train)
new_predictions=lgbm.predict(standard_test_df)
accuracy=accuracy_score(y_true=y_test,y_pred=new_predictions)
print(round(accuracy,4))

0.9375


In [10]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]
hyperparameter_grid = {'n_estimators': n_estimators,'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,'max_features': max_features}


clf=RandomizedSearchCV(ExtraTreesClassifier(random_state=1),hyperparameter_grid,cv=5,
                       n_iter=10,scoring='accuracy',n_jobs = -1,verbose = 1)

clf.fit(standard_train_df,y_train)
new_predictions=clf.predict(standard_test_df)
cnf_mat=confusion_matrix(y_true=y_test,y_pred=new_predictions,labels=['stable','unstable'])
print(cnf_mat)
accuracy=accuracy_score(y_true=y_test,y_pred=new_predictions)
print('Accuracy: {}'.format(round(accuracy*100),2))
precision=precision_score(y_true=y_test,y_pred=new_predictions, pos_label='unstable')
print('precision:{}'.format(round(precision*100),2))
recall=recall_score(y_true=y_test,y_pred=new_predictions,pos_label='unstable')
print('Recall:{}'.format(round(recall*100),2))
f1=f1_score(y_true=y_test,y_pred=new_predictions,pos_label='unstable')
print('F1:{}'.format(round(f1*100),2))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.8min finished


[[ 627   85]
 [  59 1229]]
Accuracy: 93.0
precision:94.0
Recall:95.0
F1:94.0


In [11]:
etc=ExtraTreesClassifier(random_state=1)
etc.fit(standard_train_df,y_train)
new_predictions=etc.predict(standard_test_df)
cnf_mat=confusion_matrix(y_true=y_test,y_pred=new_predictions,labels=['stable','unstable'])
print(cnf_mat)
#accuracy
accuracy=accuracy_score(y_true=y_test,y_pred=new_predictions)
print('Accuracy: {}'.format(round(accuracy*100),2))
#precision
precision=precision_score(y_true=y_test,y_pred=new_predictions, pos_label='unstable')
print('precision:{}'.format(round(precision*100),2))
#recall
recall=recall_score(y_true=y_test,y_pred=new_predictions,pos_label='unstable')
print('Recall:{}'.format(round(recall*100),2))
#
f1=f1_score(y_true=y_test,y_pred=new_predictions,pos_label='unstable')
print('F1:{}'.format(round(f1*100),2))

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[[ 613   99]
 [ 118 1170]]
Accuracy: 89.0
precision:92.0
Recall:91.0
F1:92.0


In [12]:
pd.Series(etc.feature_importances_,index=standard_train_df.columns).sort_values()

p2      0.038902
p1      0.041212
p3      0.041281
p4      0.041729
g1      0.088652
g4      0.094349
g2      0.095335
g3      0.097138
tau3    0.108774
tau4    0.114989
tau2    0.118467
tau1    0.119173
dtype: float64